# Plant Disease Classification Web App

This notebook creates a web application that:
1. Allows users to upload plant images
2. Processes the images to the required dimensions (224x224)
3. Uses a pre-trained AlexNet model to classify plant diseases
4. Displays the predicted disease class and probability

## 1. Import Required Libraries

In [18]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
from flask import Flask, render_template, request, jsonify
import io
import base64
from flask import send_from_directory

## 2. Define Disease Classes

Define the classes that your model can predict. Replace these with the actual 38 classes your model was trained on.

In [19]:
# Define the disease classes (replace with your actual 38 classes)
disease_classes = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",
    "Blueberry___healthy",
    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",
    "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Grape___healthy",
    "Orange___Haunglongbing_(Citrus_greening)",
    "Peach___Bacterial_spot",
    "Peach___healthy",
    "Pepper,_bell___Bacterial_spot",
    "Pepper,_bell___healthy",
    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",
    "Raspberry___healthy",
    "Soybean___healthy",
    "Squash___Powdery_mildew",
    "Strawberry___Leaf_scorch",
    "Strawberry___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy"
]

## 3. Load the Pre-trained Model

In [20]:
# Importing Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

# Initializing the CNN
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 38, activation = 'softmax'))
classifier.summary()


/Users/ganagraw/Desktop/ganeshk312/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 27, 27, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 17, 17, 256)    │     2,973,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 6, 6, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 6, 6, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 4, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 4, 4, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 2, 2, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 1, 1, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4096)           │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1000)           │     4,097,000 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 28,117,790 (107.26 MB)

 Trainable params: 28,096,654 (107.18 MB)

 Non-trainable params: 21,136 (82.56 KB)

In [21]:
# Load the pre-trained model using a path relative to this file
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
model_path = os.path.join(current_dir, "AlexNetModel.hdf5")
print(f"Loading model from: {model_path}")
# model = load_model(model_path)
classifier.load_weights(model_path)
model = classifier
print("Model loaded successfully!")

Loading model from: /Users/ganagraw/Desktop/ganeshk312/plant-disease/AlexNetModel.hdf5
Model loaded successfully!


## 4. Define Image Preprocessing Function

In [22]:
def preprocess_image(img_data):
    """Preprocess the uploaded image to match the model's expected input format"""
    # Load the image from data
    img = Image.open(io.BytesIO(img_data))
    
    # Resize the image to 224x224 pixels (the size expected by the model)
    img = img.resize((224, 224))
    
    # Convert the image to an array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the 4D tensor expected by the model (batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize the pixel values to between 0 and 1
    img_array = img_array / 255.0
    
    return img_array, img

## 5. Create HTML Template Directory and Files

In [23]:
# Create templates directory if it doesn't exist
os.makedirs('templates', exist_ok=True)

# Create index.html file
with open('templates/index.html', 'w') as f:
    f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>Plant Disease Classifier</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        .container {
            text-align: center;
        }
        h1 {
            color: #4CAF50;
        }
        .upload-section {
            margin: 30px 0;
            padding: 20px;
            border: 2px dashed #ccc;
            border-radius: 10px;
        }
        #preview {
            max-width: 300px;
            margin: 20px auto;
        }
        #result {
            margin: 20px 0;
            padding: 15px;
            background-color: #f1f1f1;
            border-radius: 5px;
            display: none;
        }
        .result-item {
            margin: 10px 0;
        }
        #disease-name {
            font-weight: bold;
            color: #d9534f;
            font-size: 1.2em;
        }
        button {
            background-color: #4CAF50;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Plant Disease Classifier</h1>
        <p>Upload an image of a plant leaf to identify potential diseases</p>
        
        <div class="upload-section">
            <input type="file" id="imageUpload" accept="image/*">
            <p>Supported format: JPG, JPEG, PNG</p>
        </div>
        
        <img id="preview" style="display: none;">
        
        <button id="predictBtn" style="display: none;">Analyze Disease</button>
        
        <div id="result">
            <h3>Analysis Result</h3>
            <div class="result-item">
                <p>Detected Disease: <span id="disease-name">-</span></p>
            </div>
            <div class="result-item">
                <p>Confidence: <span id="confidence">-</span></p>
            </div>
        </div>
        
        <div id="loading" style="display: none;">
            <p>Analyzing image...</p>
        </div>
    </div>
    
    <script>
        const imageUpload = document.getElementById('imageUpload');
        const preview = document.getElementById('preview');
        const predictBtn = document.getElementById('predictBtn');
        const result = document.getElementById('result');
        const diseaseName = document.getElementById('disease-name');
        const confidence = document.getElementById('confidence');
        const loading = document.getElementById('loading');
        
        imageUpload.addEventListener('change', function() {
            const file = this.files[0];
            if (file) {
                const reader = new FileReader();
                reader.onload = function(e) {
                    preview.src = e.target.result;
                    preview.style.display = 'block';
                    predictBtn.style.display = 'inline-block';
                    result.style.display = 'none';
                }
                reader.readAsDataURL(file);
            }
        });
        
        predictBtn.addEventListener('click', function() {
            const file = imageUpload.files[0];
            if (!file) return;
            
            const formData = new FormData();
            formData.append('file', file);
            
            loading.style.display = 'block';
            predictBtn.disabled = true;
            
            fetch('/predict', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                diseaseName.textContent = data.class_name;
                confidence.textContent = (data.probability * 100).toFixed(2) + '%';
                result.style.display = 'block';
                loading.style.display = 'none';
                predictBtn.disabled = false;
            })
            .catch(error => {
                console.error('Error:', error);
                loading.style.display = 'none';
                predictBtn.disabled = false;
                alert('Error analyzing image. Please try again.');
            });
        });
    </script>
</body>
</html>
''')

print("HTML template created successfully!")

HTML template created successfully!


## 6. Set Up Flask Web App

In [24]:
# Initialize Flask app
app = Flask(__name__)

# Home route
@app.route('/')
def home():
    return render_template('index.html')

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
        
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'No image selected'})
    
    try:
        # Read the image file
        img_bytes = file.read()
        
        # Preprocess the image
        img_array, original_img = preprocess_image(img_bytes)
        
        # Make prediction
        prediction = model.predict(img_array)
        
        # Get the predicted class and probability
        class_index = np.argmax(prediction[0])
        probability = float(prediction[0][class_index])
        class_name = disease_classes[class_index]
        
        # Return the result
        return jsonify({
            'class_name': class_name.replace('_', ' '),
            'probability': probability
        })
        
    except Exception as e:
        return jsonify({'error': str(e)})

## 7. Run the Flask App

In [25]:
# Run the app
if __name__ == '__main__':
    print("Starting the web server...")
    print("Access the app at http://127.0.0.1:5001")
    app.run(debug=True, port=5001, threaded=True, use_reloader=False)

Starting the web server...
Access the app at http://127.0.0.1:5001
 * Serving Flask app '__main__'
 * Debug mode: on
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2025 13:18:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2025 13:18:45] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


127.0.0.1 - - [13/Apr/2025 13:18:50] "POST /predict HTTP/1.1" 200 -


## 8. Testing the App

To test the app:

1. Run the above cell to start the server
2. Open your browser and go to http://127.0.0.1:5000
3. Upload an image of a plant leaf
4. Click "Analyze Disease" to get the prediction

Note: Make sure the AlexNetModel.hdf5 file is in the same directory as this notebook.

## 9. Creating a Standalone Python Script (Optional)

If you want to run this as a standalone application outside of this notebook, you can save the following code as `app.py`:

In [ ]:
%%writefile app.py
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
from flask import Flask, render_template, request, jsonify
import io
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

# Define the disease classes
disease_classes = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",
    "Blueberry___healthy",
    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",
    "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Grape___healthy",
    "Orange___Haunglongbing_(Citrus_greening)",
    "Peach___Bacterial_spot",
    "Peach___healthy",
    "Pepper,_bell___Bacterial_spot",
    "Pepper,_bell___healthy",
    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",
    "Raspberry___healthy",
    "Soybean___healthy",
    "Squash___Powdery_mildew",
    "Strawberry___Leaf_scorch",
    "Strawberry___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy"
]



# Initializing the CNN
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 38, activation = 'softmax'))
classifier.summary()

# Load the pre-trained model using a path relative to this script
current_dir = os.path.dirname(os.path.abspath(__file__))
model_path = os.path.join(current_dir, "AlexNetModel.hdf5")
print(f"Loading model from: {model_path}")
classifier.load_weights(model_path)
model = classifier
print("Model loaded successfully!")

def preprocess_image(img_data):
    """Preprocess the uploaded image to match the model's expected input format"""
    # Load the image from data
    img = Image.open(io.BytesIO(img_data))
    
    # Resize the image to 224x224 pixels (the size expected by the model)
    img = img.resize((224, 224))
    
    # Convert the image to an array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the 4D tensor expected by the model (batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize the pixel values to between 0 and 1
    img_array = img_array / 255.0
    
    return img_array, img

# Initialize Flask app
app = Flask(__name__)

# Home route
@app.route('/')
def home():
    return render_template('index.html')

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
        
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'No image selected'})
    
    try:
        # Read the image file
        img_bytes = file.read()
        
        # Preprocess the image
        img_array, original_img = preprocess_image(img_bytes)
        
        # Make prediction
        prediction = model.predict(img_array)
        
        # Get the predicted class and probability
        class_index = np.argmax(prediction[0])
        probability = float(prediction[0][class_index])
        class_name = disease_classes[class_index]
        
        # Return the result
        return jsonify({
            'class_name': class_name.replace('_', ' '),
            'probability': probability
        })
        
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    # Ensure templates directory exists
    os.makedirs('templates', exist_ok=True)
    
    # Run the app
    app.run(debug=True, host='0.0.0.0', port=5001)

Overwriting app.py
